# Sum types

**`Sum types`** are a way to restrict the possible states of a value to a specific set of types, making edge cases explicit.

Then we can use the **`isinstance`** function to check if an object is an instance of the subclasses that were defined as the possible types.

In the following example, I create two classes that represents two possible ways for a document: parsed and error by trying to parse. This could look strange, but in real production, a function is committed to determinate in which object should a doc be instantiated.

We use an empty parent class (`MaybeParsed` in this case) to group these types together. This acts as a conceptual label that allows type checkers and developers to know that `Parsed` and `ParseError` belong to the same family of possible results. This class could implement properties that can be inherited, like `timestamp`, for example.

This is useful because it is cleaner to represent errors as data (e.g. the `ParseError` class) rather than **`raise exceptions`** here and there. 

In [11]:
class MaybeParsed:
    pass

class Parsed(MaybeParsed):
    def __init__(self, doc_name, text):
        self.doc_name= doc_name
        self.text = text


class ParseError(MaybeParsed):
    def __init__(self, doc_name, err):
        self.doc_name = doc_name
        self.err = err

'''
Example of using isinstance later:
if isinstance(object, Parsed):
        return True
'''

'\nExample of using isinstance later:\nif isinstance(object, Parsed):\n        return True\n'

In [10]:
run_cases = [
    Parsed("why_fp.txt", "Because we're better than everyone else"),
    ParseError("why_fp.docx", "Can't handle weird windows files"),
]

submit_cases = run_cases + [
    Parsed("why_fp.md", "Because we're better than everyone else"),
    ParseError("why_fp.pdf", "Can't handle weird adobe files"),
]


def test(obj):
    print("---------------------------------")
    print(f"Testing properties of {obj.doc_name}...")
    if isinstance(obj, Parsed):
        if not obj.text:
            print(f"Expecting .text to be non-empty")
            print("Fail")
            return False
        if not obj.doc_name:
            print(f"Expecting .doc_name to be non-empty")
            print("Fail")
            return False
    elif isinstance(obj, ParseError):
        if not obj.err:
            print(f"Expecting .err to be non-empty")
            print("Fail")
            return False
        if not obj.doc_name:
            print(f"Expecting .doc_name to be non-empty")
            print("Fail")
            return False
    else:
        raise ValueError(f"unknown class type for: {obj}")
    print("Pass")
    return True


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Testing properties of why_fp.txt...
Pass
---------------------------------
Testing properties of why_fp.docx...
Pass
---------------------------------
Testing properties of why_fp.md...
Pass
---------------------------------
Testing properties of why_fp.pdf...
Pass
============= PASS ==============
4 passed, 0 failed


# Enums

Using `isinstance` works, but **`enums`** make it better. 

**`Enum`** is a class that restricts the possible values of a set of values. In the following example, the Doctype variable creates an object on the Enum class with the name **`Doctype`** and the values as second parameters. If I try to print another format (.invalid, for example), the program would raise an Exception

In [21]:
from enum import Enum

Doctype = Enum('Doctype',['PDF','TXT','DOCX','MD','HTMl'])

#The code above is the same as this one:
class Doctype(Enum): #Inheritance from the Enum class
    PDF = 1
    TXT = 2
    DOCX = 3
    MD = 4
    HTML = 5

print(Doctype.PDF)

Doctype.PDF


In [22]:
run_cases = [
    (lambda: Doctype.PDF, "Doctype.PDF", False),
    (lambda: Doctype.TXT, "Doctype.TXT", False),
    (lambda: Doctype.DOCX, "Doctype.DOCX", False),
    (lambda: Doctype.MD, "Doctype.MD", False),
]

submit_cases = run_cases + [
    (lambda: Doctype.HTML, "Doctype.HTML", False),
    (lambda: Doctype.Invalid, "Doctype.Invalid", True),
]


def test(func, name, is_err):
    print("---------------------------------")
    print(f"Checking value: {name}")
    try:
        val = func()
        print(f"...Valid enum value!")
        return not is_err
    except Exception as e:
        print(f"...Invalid enum value!")
        return is_err


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            print("Pass")
            passed += 1
        else:
            print("Fail")
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Checking value: Doctype.PDF
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.TXT
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.DOCX
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.MD
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.HTML
...Valid enum value!
Pass
---------------------------------
Checking value: Doctype.Invalid
...Invalid enum value!
Pass
============= PASS ==============
6 passed, 0 failed


Because Python does not support **`sum types`**, we should force the raise of an Exception if a value is invalid:

In [23]:
def color_to_hex(color):
    if color == Color.GREEN:
        return '#00FF00'
    elif color == Color.BLUE:
        return '#0000FF'
    elif color == Color.RED:
        return '#FF0000'
    # handle the case where the color is invalid
    raise Exception('unknown color')